## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Taltal,CL,-25.40,-70.48,60.82,65,22,4.03,few clouds
1,1,Codrington,AU,-38.27,141.97,45.86,91,100,6.71,overcast clouds
2,2,Coquimbo,CL,-29.95,-71.34,55.40,82,75,10.29,broken clouds
3,3,Butaritari,KI,3.07,172.79,83.50,72,66,19.33,broken clouds
4,4,Atuona,PF,-9.80,-139.03,77.83,75,0,16.89,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Butaritari,KI,3.07,172.79,83.50,72,66,19.33,broken clouds
4,4,Atuona,PF,-9.80,-139.03,77.83,75,0,16.89,clear sky
7,7,Port-Gentil,GA,-0.72,8.78,77.00,88,40,11.23,scattered clouds
9,9,Bubaque,GW,11.28,-15.83,81.64,85,100,5.06,overcast clouds
12,12,Vaitape,PF,-16.52,-151.75,79.20,73,3,11.65,clear sky
24,24,Arraial Do Cabo,BR,-22.97,-42.03,78.80,73,0,10.29,clear sky
28,28,Makakilo City,US,21.35,-158.09,84.20,58,1,8.05,clear sky
41,41,Sabancuy,MX,18.97,-91.18,86.32,69,98,9.44,overcast clouds
42,42,Kui Buri,TH,12.08,99.85,77.83,85,98,1.68,overcast clouds
46,46,Wajir,KE,1.75,40.06,78.85,62,63,16.22,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                224
City                   224
Country                224
Lat                    224
Lng                    224
Max Temp               224
Humidity               224
Cloudiness             224
Wind Speed             224
Current Description    224
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Butaritari,KI,83.50,broken clouds,3.07,172.79,
4,Atuona,PF,77.83,clear sky,-9.80,-139.03,
7,Port-Gentil,GA,77.00,scattered clouds,-0.72,8.78,
9,Bubaque,GW,81.64,overcast clouds,11.28,-15.83,
12,Vaitape,PF,79.20,clear sky,-16.52,-151.75,
24,Arraial Do Cabo,BR,78.80,clear sky,-22.97,-42.03,
28,Makakilo City,US,84.20,clear sky,21.35,-158.09,
41,Sabancuy,MX,86.32,overcast clouds,18.97,-91.18,
42,Kui Buri,TH,77.83,overcast clouds,12.08,99.85,
46,Wajir,KE,78.85,broken clouds,1.75,40.06,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")



Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))